In [63]:
import pandas as pd

#Import du CSV
car = pd.read_csv("dataset.csv")

#On enlève les tables qui nous sembles pas important
car = car.drop(['publishedsince', 'options', 'contrôletechnique', 'couleurextérieure', 'couleurintérieure', 'garantie', 'capacitébatterie', 'nombredepropriétaires', 'autonomiebatterie', 'conso.batterie', 'rechargeable', 'prixinclutlabatterie.1', 'voltagebatterie', 'waranty', 'puissancemoteur', 'intensitébatterie', 'prixinclutlabatterie', 'provenance', 'primeàlaconversion', 'puissancemoteur', 'vérifié&garanti', 'id', 'puissancefiscale', 'émissionsdeco2', 'consommationmixte', 'normeeuro', 'garantieconstructeur', "crit'air"], axis=1)

#Clean du dataset
car['carmodel'] = car['carmodel'].str.slice(2, -2)
car['price'] = car['price'].str.strip().str.replace('\n', '').str.replace(' ', '')
car['nombredeportes'] = car['nombredeportes'].fillna(3).astype(int)
car['nombredeplaces'] = car['nombredeplaces'].fillna(3).astype(int)
car['année'] = car['année'].fillna(0).astype(int)
car['premièremain(déclaratif)'] = car['premièremain(déclaratif)'].fillna('non')

#Clean du kilométrage
car['kilométragecompteur'] = car['kilométragecompteur'].str.replace('Km', '').str.replace(' ', '').str.strip().fillna(0).astype(int)
car = car.loc[car['kilométragecompteur'] < 250000]

#Clean de l'énergie
car['énergie'].fillna(value='NA', inplace=True)
car = car.loc[~car['énergie'].str.contains('Hybride|électrique|NA')]
car = car.loc[~car['énergie'].str.contains('Bicarburation|essence|GPL|NA')]
car = car.loc[~car['énergie'].str.contains('Carburation|essence|bioéthanol|NA')]

#Clean de la puissance
car['puissancedin'] = car['puissancedin'].str.strip().str.replace('ch', '').str.replace(' ', '')
car = car.astype({'puissancedin': 'float'})
car['puissancedin'] = car['puissancedin'].fillna(0).astype(int)
mean_power_by_model = car.groupby('carmodel')['puissancedin'].transform(lambda x: x.mean())
car.loc[car['puissancedin'] == 0, 'puissancedin'] = mean_power_by_model[car['puissancedin'] == 0]
car = car[car['puissancedin'] != 0]

#Changement de format pour la mis en circulation
car['miseencirculation'] = pd.to_datetime(car['miseencirculation'])
car['miseencirculation'] = car['miseencirculation'].dt.year

#Clean du prix
car['price'] = car['price'].str.strip().str.extract(r'([0-9 ]+)')[0].str.replace(' ', '').astype('int')
car = car.loc[car['price'] < 200000]

#Renommage des colonnes
car = car.rename(columns={'carmodel': 'model', 'price': 'prix', 'année': 'annee', 'miseencirculation' : 'mise_en_circulation','kilométragecompteur': 'kilometrage', 'énergie': 'energie', 'boîtedevitesse': 'boite', 'nombredeportes': 'nb_portes', 'nombredeplaces': 'nb_places', 'premièremain(déclaratif)': 'premiere_main', 'puissancedin': 'puissance'})

#Exportation du CSV clean
car.to_csv('../data/dataset_clean.csv', index=False)